# CSCN8020 - Assignment 1:


# Problem 1
## Pick-and-Place Robot MDP Design

### Problem Statement
Design a reinforcement learning problem as an MDP for controlling a robot arm in a repetitive pick-and-place task. The goal is to learn movements that are fast and smooth through direct motor control.

---

### MDP Components

#### States (S)
The state space captures all information necessary for effective control decisions:

- **Joint angles**: θ = (θ₁, θ₂, ..., θₙ) for each joint in the arm
- **Joint angular velocities**: θ̇ = (θ̇₁, θ̇₂, ..., θ̇ₙ) for each joint
- **End-effector position**: (x, y, z) in 3D space
- **Gripper state**: (open/closed)
- **Object status**: (held/not held)
- **Target position**: Current pick or place location

**Reasoning**: Positions tell us where the arm is located, while velocities are essential for achieving smooth motion and avoiding jerky movements. Task status information (gripper and object states) helps the agent know whether to pick or place. Since the objective emphasizes smooth movements, velocity information is crucial for the learning process.

---

#### Actions (A)
The action space consists of continuous motor commands:

- **Motor torques**: τ = (τ₁, τ₂, ..., τₙ) applied to each joint
- **Gripper command**: (open/close/hold)

**Reasoning**: Since we're controlling motors directly, actions should be torque commands or voltage signals to each motor. Continuous actions allow for smooth, precise control, which directly aligns with the smoothness objective. This low-level control gives the agent the flexibility to learn optimal trajectories.

---

#### Rewards (R)
A composite reward function that balances speed, smoothness, and task completion:

**R(s, a, s') = R_task + R_speed + R_smoothness + R_energy**

**Components:**

1. **Task Rewards (R_task)**:
   - +100 for successful object pick
   - +100 for successful object placement
   - -1 per timestep (encourages faster task completion)

2. **Speed Rewards (R_speed)**:
   - -0.1 × time_elapsed (encourages faster movements)

3. **Smoothness Rewards (R_smoothness)**:
   - -α × ||a - a_prev||² (penalizes sudden changes in motor commands)

4. **Energy Rewards (R_energy)**:
   - -β × ||a||² (penalizes excessive motor torque, encourages efficiency)

**Additional Penalties:**
- -50 for dropping the object
- -10 for collisions with workspace boundaries
- -5 for excessive joint velocities

**Reasoning**: The reward structure directly addresses the learning objectives. Task completion rewards ensure the robot fulfills its function. Time penalties encourage speed. Smoothness penalties (penalizing large changes between consecutive actions) promote smooth trajectories by discouraging abrupt motor command changes. Energy penalties prevent wasteful movements and protect the hardware. This multi-objective reward naturally guides the agent toward desired behaviors.

---

#### Transition Dynamics (P)
The transition function P(s'|s, a) is determined by the physics of the robot arm:

- **Forward dynamics equations**: How applied torques affect joint accelerations
- **Kinematic constraints**: Joint limits and workspace boundaries
- **Contact dynamics**: Gripper-object interaction physics
- **Environmental factors**: Gravity, friction, and inertia

**Reasoning**: In model-free RL, we may not explicitly model these dynamics, but the environment simulator must encode the physical laws governing robot motion. The deterministic or stochastic nature depends on factors like sensor noise and environmental uncertainty.

---

### Additional Design Considerations

1. **Discount Factor (γ)**: Use γ ≈ 0.95-0.99 to balance immediate rewards (speed) with long-term trajectory smoothness

2. **Episode Structure**: Each episode starts with the object at the pick location and ends when successfully placed at the target or after a maximum timeout

3. **State Representation**: For real implementation, continuous states would likely use function approximation (neural networks) rather than discretization

4. **Action Space**: Continuous control is preferred, potentially using algorithms like DDPG, TD3, or SAC

---

### Talking Points:

For this pick-and-place robot task, I've designed an MDP that enables learning of fast and smooth movements through direct motor control.

The **state space** includes joint angles, joint velocities, end-effector position, gripper state, and object status. Including velocities is critical because smooth motion depends on understanding movement dynamics, not just positions.

**Actions** are continuous motor torques applied to each joint, plus gripper commands. This low-level control allows the agent to learn precise, smooth trajectories.

The **reward function** balances multiple objectives: task completion bonuses (+100 for successful pick/place), time penalties to encourage speed (-1 per timestep), smoothness penalties that discourage abrupt motor command changes (-α × ||a - a_prev||²), and energy penalties to prevent excessive torques. This structure directly incentivizes the desired behavior.

**Transition dynamics** follow the physical laws governing the robot arm—how torques affect joint motion and object interactions.

This formulation provides sufficient information for learning while the reward design naturally guides the agent toward efficient, smooth pick-and-place behaviors.

---




# Problem 2
## Value Iteration on 2×2 Gridworld (Manual Calculation)

### Problem Statement
Perform two iterations of Value Iteration for a 2×2 gridworld environment. Show the step-by-step process (without code) including policy evaluation and policy improvement.

---

### Environment Setup

#### Grid Layout
```
[s1: R=5]   [s2: R=10]
[s3: R=1]   [s4: R=2]
```

#### Environment Characteristics

**State Space (S)**: {s1, s2, s3, s4}

**Action Space (A)**: {up, down, left, right}

**Initial Policy (π)**: For all states, π(up|s) = 1

**Transition Probabilities P(s'|s, a)**:
- If the action is valid (does not run into a wall), the transition is deterministic
- Otherwise, s' = s (agent stays in current state)

**Rewards R(s)**:
- R(s1) = 5 for all actions a
- R(s2) = 10 for all actions a
- R(s3) = 1 for all actions a
- R(s4) = 2 for all actions a

**Discount Factor**: γ = 0.9 (assumed)

---

### Value Iteration Algorithm

The Bellman optimality equation used for value iteration:

\[ V(s) = \max_a \sum_{s'} P(s'|s,a) [R(s) + \gamma V(s')] \]

For deterministic transitions, this simplifies to:

\[ V(s) = \max_a [R(s) + \gamma V(s')] \]

---

## Iteration 0: Initial Value Function

All states start with zero value:

| State | V₀(s) |
|-------|-------|
| s1    | 0.0   |
| s2    | 0.0   |
| s3    | 0.0   |
| s4    | 0.0   |

**Grid Visualization:**
```
[0.0]  [0.0]
[0.0]  [0.0]
```

---

## Iteration 1: First Value Update

### State s1 (Top-Left, R=5)

Calculate value for each action:

- **up**: Hits wall → stays at s1
  - V = R(s1) + γ × V₀(s1) = 5 + 0.9(0) = **5.0**

- **down**: Moves to s3
  - V = R(s1) + γ × V₀(s3) = 5 + 0.9(0) = **5.0**

- **left**: Hits wall → stays at s1
  - V = R(s1) + γ × V₀(s1) = 5 + 0.9(0) = **5.0**

- **right**: Moves to s2
  - V = R(s1) + γ × V₀(s2) = 5 + 0.9(0) = **5.0**

**V₁(s1) = max(5.0, 5.0, 5.0, 5.0) = 5.0**

---

### State s2 (Top-Right, R=10)

Calculate value for each action:

- **up**: Hits wall → stays at s2
  - V = R(s2) + γ × V₀(s2) = 10 + 0.9(0) = **10.0**

- **down**: Moves to s4
  - V = R(s2) + γ × V₀(s4) = 10 + 0.9(0) = **10.0**

- **left**: Moves to s1
  - V = R(s2) + γ × V₀(s1) = 10 + 0.9(0) = **10.0**

- **right**: Hits wall → stays at s2
  - V = R(s2) + γ × V₀(s2) = 10 + 0.9(0) = **10.0**

**V₁(s2) = max(10.0, 10.0, 10.0, 10.0) = 10.0**

---

### State s3 (Bottom-Left, R=1)

Calculate value for each action:

- **up**: Moves to s1
  - V = R(s3) + γ × V₀(s1) = 1 + 0.9(0) = **1.0**

- **down**: Hits wall → stays at s3
  - V = R(s3) + γ × V₀(s3) = 1 + 0.9(0) = **1.0**

- **left**: Hits wall → stays at s3
  - V = R(s3) + γ × V₀(s3) = 1 + 0.9(0) = **1.0**

- **right**: Moves to s4
  - V = R(s3) + γ × V₀(s4) = 1 + 0.9(0) = **1.0**

**V₁(s3) = max(1.0, 1.0, 1.0, 1.0) = 1.0**

---

### State s4 (Bottom-Right, R=2)

Calculate value for each action:

- **up**: Moves to s2
  - V = R(s4) + γ × V₀(s2) = 2 + 0.9(0) = **2.0**

- **down**: Hits wall → stays at s4
  - V = R(s4) + γ × V₀(s4) = 2 + 0.9(0) = **2.0**

- **left**: Moves to s3
  - V = R(s4) + γ × V₀(s3) = 2 + 0.9(0) = **2.0**

- **right**: Hits wall → stays at s4
  - V = R(s4) + γ × V₀(s4) = 2 + 0.9(0) = **2.0**

**V₁(s4) = max(2.0, 2.0, 2.0, 2.0) = 2.0**

---

### Updated Value Function After Iteration 1

| State | V₁(s) |
|-------|-------|
| s1    | 5.0   |
| s2    | 10.0  |
| s3    | 1.0   |
| s4    | 2.0   |

**Grid Visualization:**
```
[5.0]   [10.0]
[1.0]   [2.0]
```

**Observation**: After the first iteration, the values equal the immediate rewards since all successor state values were zero.

---

## Iteration 2: Second Value Update

Now we use V₁ values from Iteration 1 to update each state.

### State s1 (Top-Left, R=5)

Calculate value for each action:

- **up**: Hits wall → stays at s1
  - V = R(s1) + γ × V₁(s1) = 5 + 0.9(5.0) = 5 + 4.5 = **9.5**

- **down**: Moves to s3
  - V = R(s1) + γ × V₁(s3) = 5 + 0.9(1.0) = 5 + 0.9 = **5.9**

- **left**: Hits wall → stays at s1
  - V = R(s1) + γ × V₁(s1) = 5 + 0.9(5.0) = 5 + 4.5 = **9.5**

- **right**: Moves to s2
  - V = R(s1) + γ × V₁(s2) = 5 + 0.9(10.0) = 5 + 9.0 = **14.0** ✓

**V₂(s1) = max(9.5, 5.9, 9.5, 14.0) = 14.0**

**Best Action**: right (move to s2)

---

### State s2 (Top-Right, R=10)

Calculate value for each action:

- **up**: Hits wall → stays at s2
  - V = R(s2) + γ × V₁(s2) = 10 + 0.9(10.0) = 10 + 9.0 = **19.0** ✓

- **down**: Moves to s4
  - V = R(s2) + γ × V₁(s4) = 10 + 0.9(2.0) = 10 + 1.8 = **11.8**

- **left**: Moves to s1
  - V = R(s2) + γ × V₁(s1) = 10 + 0.9(5.0) = 10 + 4.5 = **14.5**

- **right**: Hits wall → stays at s2
  - V = R(s2) + γ × V₁(s2) = 10 + 0.9(10.0) = 10 + 9.0 = **19.0** ✓

**V₂(s2) = max(19.0, 11.8, 14.5, 19.0) = 19.0**

**Best Actions**: up or right (both keep agent in s2 or hit wall)

---

### State s3 (Bottom-Left, R=1)

Calculate value for each action:

- **up**: Moves to s1
  - V = R(s3) + γ × V₁(s1) = 1 + 0.9(5.0) = 1 + 4.5 = **5.5** ✓

- **down**: Hits wall → stays at s3
  - V = R(s3) + γ × V₁(s3) = 1 + 0.9(1.0) = 1 + 0.9 = **1.9**

- **left**: Hits wall → stays at s3
  - V = R(s3) + γ × V₁(s3) = 1 + 0.9(1.0) = 1 + 0.9 = **1.9**

- **right**: Moves to s4
  - V = R(s3) + γ × V₁(s4) = 1 + 0.9(2.0) = 1 + 1.8 = **2.8**

**V₂(s3) = max(5.5, 1.9, 1.9, 2.8) = 5.5**

**Best Action**: up (move to s1)

---

### State s4 (Bottom-Right, R=2)

Calculate value for each action:

- **up**: Moves to s2
  - V = R(s4) + γ × V₁(s2) = 2 + 0.9(10.0) = 2 + 9.0 = **11.0** ✓

- **down**: Hits wall → stays at s4
  - V = R(s4) + γ × V₁(s4) = 2 + 0.9(2.0) = 2 + 1.8 = **3.8**

- **left**: Moves to s3
  - V = R(s4) + γ × V₁(s3) = 2 + 0.9(1.0) = 2 + 0.9 = **2.9**

- **right**: Hits wall → stays at s4
  - V = R(s4) + γ × V₁(s4) = 2 + 0.9(2.0) = 2 + 1.8 = **3.8**

**V₂(s4) = max(11.0, 3.8, 2.9, 3.8) = 11.0**

**Best Action**: up (move to s2)

---

### Updated Value Function After Iteration 2

| State | V₂(s) | Best Action |
|-------|-------|-------------|
| s1    | 14.0  | right       |
| s2    | 19.0  | up/right    |
| s3    | 5.5   | up          |
| s4    | 11.0  | up          |

**Grid Visualization:**
```
[14.0]  [19.0]
[5.5]   [11.0]
```

---

## Summary Table

| State | V₀ | V₁  | V₂   | Optimal Action |
|-------|-----|-----|------|----------------|
| s1    | 0.0 | 5.0 | 14.0 | right          |
| s2    | 0.0 | 10.0| 19.0 | up/right       |
| s3    | 0.0 | 1.0 | 5.5  | up             |
| s4    | 0.0 | 2.0 | 11.0 | up             |

---

## Key Observations

1. **Value Propagation**: Values increase with each iteration as the agent learns to account for future rewards through the discount factor.

2. **Optimal Behavior Emerging**: After two iterations, the policy is gravitating toward state s2 (highest immediate reward of 10), with:
   - s1 → right (to s2)
   - s3 → up (to s1, then eventually to s2)
   - s4 → up (directly to s2)
   - s2 → stay (up or right keeps agent in high-reward state)

3. **Convergence**: The values are still changing, indicating that more iterations would be needed to reach the optimal value function V*.

4. **Policy Improvement**: The initial policy (all states go up) was suboptimal. After two iterations of value iteration, we've identified better actions that maximize expected cumulative reward.

---

## Mathematical Process

The value iteration process follows this pattern:

**Iteration 1**: Values reflect only immediate rewards
\[ V₁(s) = R(s) + \gamma × 0 = R(s) \]

**Iteration 2**: Values incorporate one-step lookahead
\[ V₂(s) = \max_a [R(s) + \gamma × V₁(s')] \]

**Subsequent Iterations**: Continue until convergence
\[ V_{k+1}(s) = \max_a [R(s) + \gamma × V_k(s')] \]

The algorithm converges when the maximum change in value function falls below a threshold:
\[ \max_s |V_{k+1}(s) - V_k(s)| < \theta \]

---
Talking Points:

I performed two iterations of Value Iteration on a 2×2 gridworld with rewards of 5, 10, 1, and 2 for states s1, s2, s3, and s4 respectively, using a discount factor of 0.9.

**Iteration 1** produced values equal to the immediate rewards (5.0, 10.0, 1.0, 2.0) since all successor states started at zero. **Iteration 2** showed significant value propagation: s1 increased to 14.0, s2 to 19.0, s3 to 5.5, and s4 to 11.0.

The optimal policy emerged clearly—all states gravitate toward s2, which has the highest immediate reward. State s1's best action is to move right to s2, s3 should move up toward s1 (then to s2), and s4 should move up directly to s2. State s2 itself benefits from staying put.

This demonstrates how value iteration uses the Bellman optimality equation to propagate future reward information backward through the state space, allowing the agent to discover that maximizing long-term value means navigating to and remaining in high-reward regions.

---